In [1]:
import pandas as pd

raw_data = pd.read_csv("dataset.csv")
raw_data.head()

,Title,Alternative Title,Creator,RegDate,CollectionDb,SubjectCategory,SubjectKeyword,Extent,Description,SpatialCoverage,Temporal,Person,Language,SourceTitle,ReferenceIdentifier,Rights,CopyrightOthers,URL,Contributor,SubDescription
0,"느끼다,느낌,뉘앙스",NaN,누리집,2023-05-24 11:04:42,일상생활수어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sldict.korean.go.kr/multimedia/multimed...,NaN,NaN,http://sldict.korean.go.kr/front/sign/signCont...,NaN,http://sldict.korean.go.kr/multimedia/multimed...
1,장애인 기능 경진 대회,NaN,누리집,2023-05-24 11:04:44,일상생활수어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sldict.korean.go.kr/multimedia/multimed...,NaN,NaN,http://sldict.korean.go.kr/front/sign/signCont...,NaN,http://sldict.korean.go.kr/multimedia/multimed...
2,"똑같다,같다,동일하다",NaN,누리집,2023-05-24 11:04:44,일상생활수어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sldict.korean.go.kr/multimedia/multimed...,NaN,NaN,http://sldict.korean.go.kr/front/sign/signCont...,NaN,http://sldict.korean.go.kr/multimedia/multimed...
3,"가난,곤궁,궁핍,빈곤",NaN,누리집,2023-05-24 11:04:45,일상생활수어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sldict.korean.go.kr/multimedia/multimed...,NaN,NaN,http://sldict.korean.go.kr/front/sign/signCont...,NaN,http://sldict.korean.go.kr/multimedia/multimed...
4,"반복,거듭,수시로,자꾸,자주,잦다,여러 번,연거푸",NaN,누리집,2023-05-24 11:04:45,일상생활수어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sldict.korean.go.kr/multimedia/multimed...,NaN,NaN,http://sldict.korean.go.kr/front/sign/signCont...,NaN,http://sldict.korean.go.kr/multimedia/multimed...


In [13]:
! pip install opencv-python
! pip install mediapipe
! pip install tensorflow_datasets
! pip install mecab-python3

  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl (54.8 MB)


1. Nose (코)
2. Left Eye Inner (왼쪽 눈 안쪽)
3. Left Eye (왼쪽 눈)
4. Left Eye Outer (왼쪽 눈 바깥쪽)
5. Right Eye Inner (오른쪽 눈 안쪽)
6. Right Eye (오른쪽 눈)
7. Right Eye Outer (오른쪽 눈 바깥쪽)
8. Left Ear (왼쪽 귀)
9. Right Ear (오른쪽 귀)
10. Left Shoulder (왼쪽 어깨)
11. Right Shoulder (오른쪽 어깨)
12. Left Elbow (왼쪽 팔꿈치)
13. Right Elbow (오른쪽 팔꿈치)
14. Left Wrist (왼쪽 손목)
15. Right Wrist (오른쪽 손목)
16. Left Pinky (왼쪽 새끼손가락)
17. Right Pinky (오른쪽 새끼손가락)
18. Left Index (왼쪽 검지손가락)
19. Right Index (오른쪽 검지손가락)
20. Left Thumb (왼쪽 엄지손가락)
21. Right Thumb (오른쪽 엄지손가락)
22. Left Hip (왼쪽 엉덩이)
23. Right Hip (오른쪽 엉덩이)
24. Left Knee (왼쪽 무릎)
25. Right Knee (오른쪽 무릎)
26. Left Ankle (왼쪽 발목)
27. Right Ankle (오른쪽 발목)
28. Left Heel (왼쪽 발꿈치)
29. Right Heel (오른쪽 발꿈치)
30. Left Foot Index (왼쪽 발 검지)
31. Right Foot Index (오른쪽 발 검지)
32. Left Foot (왼쪽 발)
33. Right Foot (오른쪽 발)


In [2]:
from concurrent.futures import ThreadPoolExecutor
import os
import cv2
import threading
import numpy as np
import mediapipe as mp
from tqdm import tqdm

def extract_keypoints_from_url(video_url, dest_path):
    mp_pose = mp.solutions.pose  # Define mediapipe pose model
    cap = cv2.VideoCapture(video_url)  # Use the video URL directly

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        pose_keypoints_list = []

        while True:
            opened, image = cap.read()
            if not opened:
                break

            pose_results = pose.process(image)
            if pose_results.pose_landmarks:
                pose_landmarks = pose_results.pose_landmarks
                pose_keypoints = []
                for landmark in pose_landmarks.landmark:
                    pose_keypoints.append([landmark.x, landmark.y, landmark.z])
                pose_keypoints_list.append(pose_keypoints)

        pose_keypoints_array = np.array(pose_keypoints_list)
        np.save(f'{dest_path}/{os.path.basename(video_url)}_p.npy', pose_keypoints_array)

    cap.release()

In [3]:
output_path = './keypoint'

In [4]:
test_X = raw_data["SubDescription"].to_list()[0]
path = './keypoint'
extract_keypoints_from_url(test_X, path)

I0000 00:00:1734961667.339539 12283065 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1734961667.435618 12285397 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734961667.451253 12285391 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734961667.470818 12285395 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [5]:
def working_threads():
    executor = ThreadPoolExecutor(max_workers=8)
    futures = []

    # 비디오 URL을 직접 처리하도록 수정
    video_urls = raw_data["SubDescription"].to_list()

    for video_url in tqdm(video_urls, colour='green'):
        future = executor.submit(extract_keypoints_from_url, video_url, output_path)
        futures.append(future)

    print("file listup finish.. threads start..")

    for future in tqdm(futures, colour='blue'):
        future.result()

    executor.shutdown()

working_threads()

100%|██████████| 3634/3634 [00:00<00:00, 187079.32it/s]


file listup finish.. threads start..


  0%|          | 0/3634 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1734962446.068150 12316073 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
I0000 00:00:1734962446.086597 12316071 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1734962446.251448 12316091 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734962446.251448 12316115 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734962446.286037 12316115 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734962446.289338 12316091 inf

In [4]:
import numpy as np

# npy 파일 불러오기
def load_keypoints_from_npy(file_path):
    # .npy 파일 로드
    keypoints_array = np.load(file_path)
    
    # 로드한 배열의 형태 확인
    if keypoints_array.ndim == 3 and keypoints_array.shape[1:] == (33, 3):
        print(f"Loaded array with shape: {keypoints_array.shape}")
    else:
        raise ValueError(f"Unexpected shape: {keypoints_array.shape}. Expected (N, 33, 3).")
    
    return keypoints_array

# 사용 예시
file_path = './keypoint/MOV000361363_700X466.mp4_p.npy'
keypoints_array = load_keypoints_from_npy(file_path)
print(keypoints_array)

Loaded array with shape: (231, 33, 3)
[[[ 0.50974232  0.27930358 -0.55487412]
  [ 0.52328157  0.24937269 -0.52061677]
  [ 0.52983117  0.24950999 -0.52089173]
  ...
  [ 0.43886155  1.45425594  0.63904923]
  [ 0.53349668  1.51047933  0.3924832 ]
  [ 0.45125818  1.50809753  0.37588814]]

 [[ 0.50992352  0.28048235 -0.55147153]
  [ 0.5232836   0.25065508 -0.51669979]
  [ 0.52985108  0.2508769  -0.51700473]
  ...
  [ 0.44429705  1.46459067  0.61702961]
  [ 0.53832221  1.52563131  0.34388885]
  [ 0.45549226  1.51432192  0.35559782]]

 [[ 0.51018041  0.28099683 -0.55159622]
  [ 0.52330351  0.25109518 -0.51678491]
  [ 0.52990013  0.25132748 -0.51708555]
  ...
  [ 0.44656354  1.47334373  0.62060398]
  [ 0.5429247   1.5364641   0.35142219]
  [ 0.45917743  1.52192783  0.35807857]]

 ...

 [[ 0.51034552  0.27814409 -0.55040294]
  [ 0.52125221  0.24944898 -0.51252639]
  [ 0.52786845  0.24974836 -0.51285243]
  ...
  [ 0.448587    1.5186125   0.47482613]
  [ 0.55135679  1.58923495  0.28554055]
  [ 0.

In [6]:
import numpy as np

def load_npy_files_from_directory(directory_path):
    npy_files = [f for f in os.listdir(directory_path) if f.endswith('.npy')]  # .npy 파일만 필터링
    npy_data = []

    for npy_file in npy_files:
        npy_path = os.path.join(directory_path, npy_file)
        data = np.load(npy_path)  # npy 파일 로드
        npy_data.append(data)

    return npy_data

directory_path = output_path # 실제 디렉터리 경로로 변경
npy_data_list = load_npy_files_from_directory(directory_path)
npy_data_list[:5]

[array([[[ 0.5003593 ,  0.27420294, -0.69937778],
         [ 0.51598543,  0.24197017, -0.66817087],
         [ 0.52351105,  0.24199365, -0.66839957],
         ...,
         [ 0.46006933,  1.48291266,  0.61975968],
         [ 0.56385851,  1.52997696,  0.35193172],
         [ 0.46667257,  1.53144825,  0.33953658]],
 
        [[ 0.50045198,  0.27381203, -0.71106791],
         [ 0.51600182,  0.24193621, -0.67894077],
         [ 0.52365297,  0.2419939 , -0.6791805 ],
         ...,
         [ 0.45763698,  1.50211203,  0.68874377],
         [ 0.5601446 ,  1.55507672,  0.34520081],
         [ 0.46311375,  1.55164683,  0.40563318]],
 
        [[ 0.50051773,  0.2732749 , -0.7100783 ],
         [ 0.51600116,  0.24176282, -0.6780507 ],
         [ 0.52370858,  0.24191271, -0.67828888],
         ...,
         [ 0.45775416,  1.50987387,  0.6848101 ],
         [ 0.56020314,  1.56372607,  0.3456856 ],
         [ 0.46318656,  1.56101024,  0.4016363 ]],
 
        ...,
 
        [[ 0.50287062,  0.27214673

In [29]:
import numpy as np
import os

# 사용 예시
directory_path = './keypoint'
keypoints_list = []
name_list = []
title_list = []

# 가장 긴 frames 길이를 찾기 위한 변수
max_frames = 0
    
# 디렉토리 내 모든 npy 파일 순회
for filename in os.listdir(directory_path):
    if filename.endswith(".npy"):
        file_path = os.path.join(directory_path, filename)
        keypoints_array = np.load(file_path)
            
        # 배열의 형식 확인
        if keypoints_array.ndim == 3 and keypoints_array.shape[1:] == (33, 3):
            keypoints_list.append(keypoints_array)
            name_list.append(filename)
            # 가장 긴 frames 길이 업데이트
            max_frames = max(max_frames, keypoints_array.shape[0])
            print(f"Loaded: {file_path} with shape {keypoints_array.shape}")
        else:
            print(f"Skipped: {file_path} due to unexpected shape {keypoints_array.shape}")

# 각 배열을 가장 긴 frames 길이에 맞게 패딩
padded_keypoints_list = []
for keypoints_array in keypoints_list:
    frames_to_add = max_frames - keypoints_array.shape[0]
    if frames_to_add > 0:
        # 패딩: 0으로 채워서 같은 크기로 만듦
        padding = np.zeros((frames_to_add, 33, 3))
        padded_keypoints_array = np.vstack([keypoints_array, padding])
    else:
        padded_keypoints_array = keypoints_array
    padded_keypoints_list.append(padded_keypoints_array)

# padded_keypoints_list를 np.array로 변환
all_keypoints = np.array(padded_keypoints_list)

# 변환된 keypoints 배열의 형태 출력
print(f"All keypoints array shape: {all_keypoints.shape}")

Loaded: ./keypoint/MOV000241109_700X466.mp4_p.npy with shape (198, 33, 3)
Loaded: ./keypoint/MOV000360840_700X466.mp4_p.npy with shape (251, 33, 3)
Loaded: ./keypoint/MOV000260294_700X466.mp4_p.npy with shape (177, 33, 3)
Loaded: ./keypoint/MOV000249064_700X466.mp4_p.npy with shape (189, 33, 3)
Loaded: ./keypoint/MOV000235403_700X466.mp4_p.npy with shape (244, 33, 3)
Loaded: ./keypoint/MOV000235399_700X466.mp4_p.npy with shape (260, 33, 3)
Loaded: ./keypoint/MOV000359513_700X466.mp4_p.npy with shape (241, 33, 3)
Loaded: ./keypoint/MOV000240345_700X466.mp4_p.npy with shape (259, 33, 3)
Loaded: ./keypoint/MOV000250301_700X466.mp4_p.npy with shape (188, 33, 3)
Loaded: ./keypoint/MOV000241147_700X466.mp4_p.npy with shape (111, 33, 3)
Loaded: ./keypoint/MOV000359847_700X466.mp4_p.npy with shape (241, 33, 3)
Loaded: ./keypoint/MOV000360073_700X466.mp4_p.npy with shape (211, 33, 3)
Loaded: ./keypoint/MOV000252375_700X466.mp4_p.npy with shape (260, 33, 3)
Loaded: ./keypoint/MOV000253177_700X46

In [3]:
print(len(all_keypoints))
print(len(name_list))


3609
3609


In [21]:
len(raw_data)

3634

In [8]:
import os

def match_and_get_titles(npy_files_name, raw_data):
    y = []

    for name in npy_files_name:
        # npy 파일 이름에 매칭되는 행을 loc로 검색
        matched_row = raw_data.loc[raw_data["SubDescription"].apply(lambda d: os.path.basename(d).strip() + "_p.npy") == name]
        
        if not matched_row.empty:
            y.append(matched_row.iloc[0]["Title"])  # 첫 번째 매칭된 행의 Title 추가
    
    return y

In [18]:
def match_and_get_titles(npy_files_name, raw_data):
    y = []

    for name in npy_files_name:

        for _, row in raw_data.iterrows(): 
            d = row["SubDescription"]  # SubDescription 열의 값을 가져옴
            d_base_name = os.path.basename(d).strip() + "_p.npy"

            if name == d_base_name:
                y.append(row["Title"])
                break
    
    return y



In [40]:
title_list = match_and_get_titles(name_list, raw_data)
title_list

['바람나다,바람',
 '수의사,수의',
 '갚았다',
 '낙담,낙망,낙심,실망,절망,좌절',
 '지난주,작주,전주',
 '지갑',
 '특수교육',
 '따로,분리하다',
 '깨다',
 '마찬가지,같다,동일,맞다,동,똑같다',
 '뻐기다',
 '천,일천',
 '말,승마',
 '법,법규,법률,법칙',
 '던지다,투수',
 '곤두박질,곤두질,떨어지다',
 '보류,유보',
 '은메달',
 '아름답다,미',
 '하나님,하느님',
 '자식,자녀',
 '콩',
 '반장,급장',
 '남해',
 '사무직',
 '신기하다,교묘하다,묘하다,오묘하다,이상',
 '부두,항구,항',
 '골프',
 '평안북도',
 '지옥,나락',
 '방학',
 '색칠,칠,도색',
 '신나다',
 '신도시',
 '(혈액의)순환',
 '혈액형,혈액군,혈형',
 '복판,가운데,중심,한가운데',
 '휘어잡다,지배',
 '개나리',
 '소외,이탈',
 '운동,스포츠,체육',
 '센터',
 '똑똑하다,영리하다',
 '가마니,가마',
 '중화요리,청요리',
 '잔디,풀',
 '사기꾼',
 '방',
 '과제,숙제',
 '축구장',
 '그만,끝,까지,종결,종료,끝나다,마치다',
 '내리누르다',
 '외투,겉옷',
 '검도',
 '농사,농사일,농업',
 '횡재',
 '국가시험,국가 고시',
 '동반자',
 '글월,문,문구,문장',
 '팔도',
 '며느리,아가,자부',
 '부자,자산가,재산가',
 '원금,자금,원가,본전',
 '무찌르다',
 '피의자,용의자',
 '(열매)밤',
 '곧,금명간,신속,이르다,냉큼,빨리,속히,썩,어서,얼른,이내,일찍,일찍이,-자',
 '양초,초',
 '설명',
 '영웅',
 '일출,동,아침,(해가)돋다',
 '판결의 공시',
 '내복,내의,속옷',
 '운동장',
 '자기,스스로,자체,혼자,홀로,자발,자신',
 '고조할머니',
 '요일',
 '법정,재판정',
 '뱃사람,선원',
 '공항,비행장',
 '산악',
 '노비,노예,심부름',
 '국회의사당',
 '열흘,열흘날',
 '발레',
 '자음',

In [41]:
len(title_list)

3609

In [42]:
y = title_list
y

['바람나다,바람',
 '수의사,수의',
 '갚았다',
 '낙담,낙망,낙심,실망,절망,좌절',
 '지난주,작주,전주',
 '지갑',
 '특수교육',
 '따로,분리하다',
 '깨다',
 '마찬가지,같다,동일,맞다,동,똑같다',
 '뻐기다',
 '천,일천',
 '말,승마',
 '법,법규,법률,법칙',
 '던지다,투수',
 '곤두박질,곤두질,떨어지다',
 '보류,유보',
 '은메달',
 '아름답다,미',
 '하나님,하느님',
 '자식,자녀',
 '콩',
 '반장,급장',
 '남해',
 '사무직',
 '신기하다,교묘하다,묘하다,오묘하다,이상',
 '부두,항구,항',
 '골프',
 '평안북도',
 '지옥,나락',
 '방학',
 '색칠,칠,도색',
 '신나다',
 '신도시',
 '(혈액의)순환',
 '혈액형,혈액군,혈형',
 '복판,가운데,중심,한가운데',
 '휘어잡다,지배',
 '개나리',
 '소외,이탈',
 '운동,스포츠,체육',
 '센터',
 '똑똑하다,영리하다',
 '가마니,가마',
 '중화요리,청요리',
 '잔디,풀',
 '사기꾼',
 '방',
 '과제,숙제',
 '축구장',
 '그만,끝,까지,종결,종료,끝나다,마치다',
 '내리누르다',
 '외투,겉옷',
 '검도',
 '농사,농사일,농업',
 '횡재',
 '국가시험,국가 고시',
 '동반자',
 '글월,문,문구,문장',
 '팔도',
 '며느리,아가,자부',
 '부자,자산가,재산가',
 '원금,자금,원가,본전',
 '무찌르다',
 '피의자,용의자',
 '(열매)밤',
 '곧,금명간,신속,이르다,냉큼,빨리,속히,썩,어서,얼른,이내,일찍,일찍이,-자',
 '양초,초',
 '설명',
 '영웅',
 '일출,동,아침,(해가)돋다',
 '판결의 공시',
 '내복,내의,속옷',
 '운동장',
 '자기,스스로,자체,혼자,홀로,자발,자신',
 '고조할머니',
 '요일',
 '법정,재판정',
 '뱃사람,선원',
 '공항,비행장',
 '산악',
 '노비,노예,심부름',
 '국회의사당',
 '열흘,열흘날',
 '발레',
 '자음',

In [45]:
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from tensorflow.keras.utils import to_categorical


# 1. 쉼표(,)로 구분하여 레이블 분리
y_split = [item.split(',') for item in y]

model = Word2Vec(sentences=y_split, vector_size=100, window=5, min_count=1, workers=4)

# 3. 각 레이블을 벡터로 변환
def get_vector_for_label(label):
    # 각 레이블에 대해 단어 벡터를 평균하여 하나의 벡터로 만듬
    word_vectors = [model.wv[word] for word in label if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)  # 평균 벡터 반환
    else:
        return np.zeros(model.vector_size)  # 벡터가 없으면 0으로 채운 벡터 반환

# 4. y 데이터를 벡터화 (각 레이블에 대해 벡터 반환)
y_vectors = np.array([get_vector_for_label(labels) for labels in y_split])
y_vectors

array([[-0.00076176,  0.00630615, -0.00099506, ...,  0.00687233,
         0.0020429 ,  0.00176714],
       [ 0.00815635,  0.0060722 ,  0.00370319, ...,  0.00134445,
        -0.00213488, -0.00086652],
       [-0.00487208, -0.00037248, -0.00431124, ...,  0.00539282,
        -0.00469651, -0.00236953],
       ...,
       [-0.00027849,  0.00835625, -0.00306663, ...,  0.00244025,
        -0.00160929, -0.00237284],
       [ 0.00218083, -0.00120573,  0.00224365, ..., -0.00155841,
         0.00404155,  0.00304336],
       [ 0.00441491, -0.00101647, -0.00126488, ..., -0.00166213,
         0.00382152, -0.0013747 ]], dtype=float32)

In [71]:
y_vectors.shape

(3609, 100)

In [107]:
from sklearn.model_selection import train_test_split

X = all_keypoints
X_train, X_test, y_train, y_test = train_test_split(X, y_vectors, test_size=0.2, random_state=42)

In [108]:
# 4D 데이터를 3D로 변환
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)

# 새로운 입력 형태 확인
print("X_train_reshaped shape:", X_train_reshaped.shape)
print("X_test_reshaped shape:", X_test_reshaped.shape)


X_train_reshaped shape: (2887, 407, 99)
X_test_reshaped shape: (722, 407, 99)


In [109]:
X_train = X_train_reshaped
X_test = X_test_reshaped

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2887, 407, 99) (2887, 100)
(722, 407, 99) (722, 100)


In [110]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

# 데이터 준비: 입력 데이터의 차원을 CNN에 맞게 변경
X_train = tf.expand_dims(X_train, axis=-1)  # (2887, 407, 99, 1)
X_test = tf.expand_dims(X_test, axis=-1)    # (722, 407, 99, 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# CNN 모델 구축
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), input_shape=(407, 99, 1), activation='relu'),  # 입력 크기 맞춤
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='softmax')  # 출력 노드 수 100으로 설정
])

# 모델 요약
model.summary()

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200)


(2887, 407, 99, 1) (2887, 100)
(722, 407, 99, 1) (722, 100)


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_26 (Conv2D)              │ (None, 405, 97, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 202, 48, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 202, 48, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 200, 46, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 100, 23, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 147200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 128)            │    18,841,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,873,444 (72.00 MB)

 Trainable params: 18,873,444 (72.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 640ms/step - accuracy: 0.0119 - loss: -0.0014 - val_accuracy: 0.0111 - val_loss: 0.0200
Epoch 2/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 71s 784ms/step - accuracy: 0.0115 - loss: -9.7867 - val_accuracy: 0.0111 - val_loss: 0.1242
Epoch 3/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 46s 504ms/step - accuracy: 0.0097 - loss: -693.2422 - val_accuracy: 0.0111 - val_loss: 424.7601
Epoch 4/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 49s 535ms/step - accuracy: 0.0107 - loss: -2742.4600 - val_accuracy: 0.0097 - val_loss: 350.2915
Epoch 5/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 60s 651ms/step - accuracy: 0.0086 - loss: -7966.9863 - val_accuracy: 0.0055 - val_loss: 2624.2734
Epoch 6/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 46s 501ms/step - accuracy: 0.0111 - loss: -6757.5664 - val_accuracy: 0.0111 - val_loss: 2928.6648
Epoch 7/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 54s 598ms/step - accuracy: 0.0139 - loss: -14079.7812 - val_accuracy: 0.0097 - val_loss: 1523.4628
Epoch 8/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 71s 778ms/step -

KeyboardInterrupt: 

In [103]:

np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

# custom f1 score
def metric_F1score(y_true,y_pred): 
    TP=tf.reduce_sum(y_true*tf.round(y_pred))
    TN=tf.reduce_sum((1-y_true)*(1-tf.round(y_pred)))
    FP=tf.reduce_sum((1-y_true)*tf.round(y_pred))
    FN=tf.reduce_sum(y_true*(1-tf.round(y_pred)))
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    F1score=2*precision*recall/(precision+recall)
    return F1score

In [104]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras as keras

model = Sequential([
    LSTM(64, activation='relu', input_shape=X_train.shape[1:3], kernel_regularizer=keras.regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(100, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc',metric_F1score])
model.summary()

/opt/anaconda3/envs/dipLearning/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                   │ (None, 64)             │        41,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 100)            │         3,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,364 (185.02 KB)

 Trainable params: 47,364 (185.02 KB)

 Non-trainable params: 0 (0.00 B)

In [105]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=200,
)

Epoch 1/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - acc: 0.0130 - loss: nan - metric_f1score: nan - val_acc: 0.0139 - val_loss: nan - val_metric_f1score: nan
Epoch 2/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - acc: 0.0089 - loss: nan - metric_f1score: nan - val_acc: 0.0139 - val_loss: nan - val_metric_f1score: nan
Epoch 3/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 89ms/step - acc: 0.0089 - loss: nan - metric_f1score: nan - val_acc: 0.0139 - val_loss: nan - val_metric_f1score: nan
Epoch 4/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 89ms/step - acc: 0.0089 - loss: nan - metric_f1score: nan - val_acc: 0.0139 - val_loss: nan - val_metric_f1score: nan
Epoch 5/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 88ms/step - acc: 0.0089 - loss: nan - metric_f1score: nan - val_acc: 0.0139 - val_loss: nan - val_metric_f1score: nan
Epoch 6/200
55/91 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - acc: 0.0078 - loss: nan - metric_f1score: nan

KeyboardInterrupt: 

In [17]:
tokenized_titles = [title.split(',') for title in title_list]
tokenized_titles

[['바람나다', '바람'],
 ['수의사', '수의'],
 ['갚았다'],
 ['낙담', '낙망', '낙심', '실망', '절망', '좌절'],
 ['지난주', '작주', '전주'],
 ['지갑'],
 ['특수교육'],
 ['따로', '분리하다'],
 ['깨다'],
 ['마찬가지', '같다', '동일', '맞다', '동', '똑같다'],
 ['뻐기다'],
 ['천', '일천'],
 ['말', '승마'],
 ['법', '법규', '법률', '법칙'],
 ['던지다', '투수'],
 ['곤두박질', '곤두질', '떨어지다'],
 ['보류', '유보'],
 ['은메달'],
 ['아름답다', '미'],
 ['하나님', '하느님'],
 ['자식', '자녀'],
 ['콩'],
 ['반장', '급장'],
 ['남해'],
 ['사무직'],
 ['신기하다', '교묘하다', '묘하다', '오묘하다', '이상'],
 ['부두', '항구', '항'],
 ['골프'],
 ['평안북도'],
 ['지옥', '나락'],
 ['방학'],
 ['색칠', '칠', '도색'],
 ['신나다'],
 ['신도시'],
 ['(혈액의)순환'],
 ['혈액형', '혈액군', '혈형'],
 ['복판', '가운데', '중심', '한가운데'],
 ['휘어잡다', '지배'],
 ['개나리'],
 ['소외', '이탈'],
 ['운동', '스포츠', '체육'],
 ['센터'],
 ['똑똑하다', '영리하다'],
 ['가마니', '가마'],
 ['중화요리', '청요리'],
 ['잔디', '풀'],
 ['사기꾼'],
 ['방'],
 ['과제', '숙제'],
 ['축구장'],
 ['그만', '끝', '까지', '종결', '종료', '끝나다', '마치다'],
 ['내리누르다'],
 ['외투', '겉옷'],
 ['검도'],
 ['농사', '농사일', '농업'],
 ['횡재'],
 ['국가시험', '국가 고시'],
 ['동반자'],
 ['글월', '문', '문구', '문장'],
 ['팔도'],
 ['며느리', '아가', '